# Aggregation for Excel visualisation

This notebook combines sentence-level predictions with page counts, aggregates by company/year/label, and computes metrics across multiple probability thresholds.

In [9]:
import numpy as np
import pandas as pd
from pathlib import Path


## Load combined predictions

`combine_csvs.py` output should be in `outputs/final_result.csv`.

In [10]:
preds_path = Path("outputs/final_result.csv")
df = pd.read_csv(preds_path)
df.head()

,sentence,source_page,source_pdf,prob_fin_label,prob_soc_label,prob_env_label,prob_maori_label,company,year
0,29 August 2013 The Manager ASX Market Announce...,1,AFI2013,0.003772,0.092093,0.707032,0.000960,AFI,2013
1,Yours faithfully Simon Pordage Company Secretary,1,AFI2013,0.002400,0.994174,0.003048,0.001599,AFI,2013
2,EXPERIENCE INCOME GROWTH Annual Report 2013,2,AFI2013,0.999252,0.005513,0.003998,0.001436,AFI,2013
3,"To pay dividends which, over time, grow faster...",3,AFI2013,0.998418,0.001096,0.027033,0.001041,AFI,2013
4,2013 2012 2011 2010 2009 Net profit after tax ...,4,AFI2013,0.193524,0.000989,0.940657,0.000736,AFI,2013


## Reshape to long format (label + probability) and compute word counts

In [11]:
label_map = {
    "prob_fin_label": "fin",
    "prob_soc_label": "soc",
    "prob_env_label": "env",
    "prob_maori_label": "maori",
}

df["word_count"] = df["sentence"].astype(str).str.split().str.len()

long_df = df.melt(
    id_vars=["company", "year", "word_count"],
    value_vars=list(label_map.keys()),
    var_name="label",
    value_name="probability",
)
long_df["label"] = long_df["label"].map(label_map)
long_df = long_df.dropna(subset=["label", "probability"])
long_df.head()


,company,year,word_count,label,probability
0,AFI,2013,65,fin,0.003772
1,AFI,2013,6,fin,0.002400
2,AFI,2013,6,fin,0.999252
3,AFI,2013,13,fin,0.998418
4,AFI,2013,87,fin,0.193524


## Basic checks and derive company/year from source_pdf

Each CSV represents one company-year, so we ignore existing company/year columns and parse from `source_pdf` (e.g., AFI2013 -> AFI, 2013).

In [12]:
required_cols = {
    "sentence",
    "company",
    "year",
    "prob_fin_label",
    "prob_soc_label",
    "prob_env_label",
    "prob_maori_label",
}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"Missing required columns: {missing}")

df["year"] = pd.to_numeric(df["year"], errors="coerce").astype("Int64")
df = df.dropna(subset=["company", "year", "sentence"])
df.head()


,sentence,source_page,source_pdf,prob_fin_label,prob_soc_label,prob_env_label,prob_maori_label,company,year,word_count
0,29 August 2013 The Manager ASX Market Announce...,1,AFI2013,0.003772,0.092093,0.707032,0.000960,AFI,2013,65
1,Yours faithfully Simon Pordage Company Secretary,1,AFI2013,0.002400,0.994174,0.003048,0.001599,AFI,2013,6
2,EXPERIENCE INCOME GROWTH Annual Report 2013,2,AFI2013,0.999252,0.005513,0.003998,0.001436,AFI,2013,6
3,"To pay dividends which, over time, grow faster...",3,AFI2013,0.998418,0.001096,0.027033,0.001041,AFI,2013,13
4,2013 2012 2011 2010 2009 Net profit after tax ...,4,AFI2013,0.193524,0.000989,0.940657,0.000736,AFI,2013,87


## Load and aggregate page counts

`merge_page_counts.py` output should be in `outputs/page_counts_merged.csv`. Pages are summed per company/year.

In [13]:
pages_path = Path("outputs/page_counts_merged.csv")
pages = pd.read_csv(pages_path)

required_pages = {"company", "year", "pages"}
missing_pages = required_pages - set(pages.columns)
if missing_pages:
    raise ValueError(f"Missing page count columns: {missing_pages}")

pages["year"] = pd.to_numeric(pages["year"], errors="coerce").astype("Int64")
pages["pages"] = pd.to_numeric(pages["pages"], errors="coerce")
page_counts = (
    pages.dropna(subset=["company", "year", "pages"])
    .groupby(["company", "year"], as_index=False)["pages"]
    .sum()
)
page_counts.head()

,company,year,pages
0,AFI,2013,143
1,AFI,2014,140
2,AFI,2015,125
3,AFI,2016,123
4,AFI,2017,80


## Aggregation helper

In [14]:
BASE_YEAR = int(df["year"].min())

def summarize_over_threshold(df_long: pd.DataFrame, threshold: float) -> pd.DataFrame:
    over_mask = df_long["probability"] > threshold
    enriched = df_long.assign(
        over=over_mask,
        over_word_count=lambda d: d["word_count"].where(over_mask, 0),
    )

    agg = (
        enriched
        .groupby(["company", "year", "label"])
        .agg(
            total_sentences=("probability", "size"),
            total_words=("word_count", "sum"),
            over_count=("over", "sum"),
            over_share=("over", "mean"),
            over_word_count=("over_word_count", "sum"),
            mean_prob_over=("probability", lambda s: s[s > threshold].mean() if (s > threshold).any() else 0.0),
            sum_prob_over=("probability", lambda s: s[s > threshold].sum()),
        )
        .reset_index()
    )

    agg = agg.merge(page_counts, on=["company", "year"], how="left")
    agg["pages"] = agg["pages"].replace({0: pd.NA})

    agg["over_sentence_word_share"] = agg["over_count"] / agg["total_words"]
    agg["over_word_share"] = agg["over_word_count"] / agg["total_words"]
    agg["sentences_per_page"] = agg["total_sentences"] / agg["pages"]
    agg["over_sentences_per_page"] = agg["over_count"] / agg["pages"]
    agg["over_words_per_page"] = agg["over_word_count"] / agg["pages"]

    agg["theme_rate_per_1000_words"] = np.where(
        agg["total_words"] > 0,
        (agg["over_count"] / agg["total_words"]) * 1000,
        0.0,
    )
    non_theme = agg["total_sentences"] - agg["over_count"]
    agg["theme_ratio_per_1000_sentences"] = np.where(
        non_theme > 0,
        (agg["over_count"] / non_theme) * 1000,
        0.0,
    )

    base_counts = (
        agg.loc[agg["year"] == BASE_YEAR, ["label", "company", "over_count"]]
        .drop_duplicates(subset=["label", "company"])
        .set_index(["label", "company"])["over_count"]
    )
    agg["theme_index_base"] = agg.apply(
        lambda row: (row["over_count"] / base_counts.get((row["label"], row["company"]), 0)) * 100
        if base_counts.get((row["label"], row["company"]), 0) > 0 else 0.0,
        axis=1,
    )

    agg["avg_theme_score_relevant"] = agg["mean_prob_over"]
    agg["threshold"] = threshold
    return agg


## Output column definitions
The aggregation output includes the following columns:

```text
# company: company identifier
# year: reporting year
# label: theme/label for the sentence classification
# total_sentences: number of sentences in the company/year/label group
# total_words: total word_count in the group
# over_count: count of sentences with probability > threshold
# over_share: share of sentences over threshold (over_count / total_sentences)
# over_word_count: total word_count for sentences over threshold
# mean_prob_over: mean probability among sentences over threshold (0 if none)
# sum_prob_over: sum of probabilities among sentences over threshold
# pages: total pages for the company/year (0 replaced with NA)
# over_sentence_word_share: over_count / total_words
# over_word_share: over_word_count / total_words
# sentences_per_page: total_sentences / pages
# over_sentences_per_page: over_count / pages
# over_words_per_page: over_word_count / pages
# theme_rate_per_1000_words: (over_count / total_words) * 1000
# theme_ratio_per_1000_sentences: (over_count / (total_sentences - over_count)) * 1000
# theme_index_base: over_count indexed to base year per company/label (base=100)
# avg_theme_score_relevant: alias of mean_prob_over
# threshold: probability threshold used for this row
```


## Build summaries for all thresholds

In [15]:
thresholds = [0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
summaries = [summarize_over_threshold(long_df, thr) for thr in thresholds]
summary_all = pd.concat(summaries, ignore_index=True)
summary_all.head()


,company,year,label,total_sentences,total_words,over_count,over_share,over_word_count,mean_prob_over,sum_prob_over,...,over_sentence_word_share,over_word_share,sentences_per_page,over_sentences_per_page,over_words_per_page,theme_rate_per_1000_words,theme_ratio_per_1000_sentences,theme_index_base,avg_theme_score_relevant,threshold
0,AFI,2013,env,1281,35255,259,0.202186,7858,0.950899,246.282830,...,0.007346,0.222890,8.958042,1.811189,54.951049,7.346476,253.424658,0.0,0.950899,0.7
1,AFI,2013,fin,1281,35255,508,0.396565,15362,0.968856,492.178661,...,0.014409,0.435740,8.958042,3.552448,107.426573,14.409304,657.179819,0.0,0.968856,0.7
2,AFI,2013,maori,1281,35255,0,0.000000,0,0.000000,0.000000,...,0.000000,0.000000,8.958042,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.7
3,AFI,2013,soc,1281,35255,364,0.284153,8886,0.952104,346.565956,...,0.010325,0.252049,8.958042,2.545455,62.139860,10.324777,396.946565,0.0,0.952104,0.7
4,AFI,2014,env,1208,33582,243,0.201159,7591,0.947982,230.359732,...,0.007236,0.226044,8.628571,1.735714,54.221429,7.236019,251.813472,0.0,0.947982,0.7


## Save for Excel

In [16]:
output_path = Path("outputs/aggregation_thresholds.csv")
output_path.parent.mkdir(parents=True, exist_ok=True)
summary_all.to_csv(output_path, index=False)
print(f"Saved: {output_path}")

Saved: outputs/aggregation_thresholds.csv
